In [1]:
import trimesh
import matplotlib.pyplot as plt
import numpy as np
import random
import os
from tqdm import tqdm

In [2]:
def get_voxel_matrix(mesh):
    # Perform voxelization and convert to float32 directly
    voxel = mesh.voxelized(pitch=1.0 / 64)
    voxel_matrix = voxel.matrix.astype(np.float32)

    # Check dimensions of voxel_matrix
    if voxel_matrix.shape[0] > 256 or voxel_matrix.shape[1] > 256 or voxel_matrix.shape[2] > 256:
        return [None]

    # Initialize a padded matrix with zeros and copy voxel_matrix into it
    padded_matrix = np.zeros((256, 256, 256), dtype=np.float32)
    padded_matrix[:voxel_matrix.shape[0], :voxel_matrix.shape[1], :voxel_matrix.shape[2]] = voxel_matrix

    # Directly create the VoxelGrid from the padded matrix
    padded_voxelized = trimesh.voxel.VoxelGrid(padded_matrix)
    
    return padded_voxelized.matrix.astype(np.float32)

def create_view(voxel_matrix):
    # Use np.moveaxis for better performance instead of separate lists
    # This reduces the number of loops and utilizes NumPy's efficient operations
    views = np.moveaxis(voxel_matrix, 0, -1)

    # Prepare the first_view with shape (256, 256, 256, 3)
    first_view = np.zeros((256, 256, 256, 3), dtype=np.float32)
    first_view[..., 0] = views[:, :, 0]  # Axial
    first_view[..., 1] = views[:, 0, :]  # Sagittal
    first_view[..., 2] = views[0, :, :]  # Coronal
    
    return first_view

In [3]:
"""os.makedirs('voxel_dataset/train/', exist_ok=True)
os.makedirs('voxel_dataset/ground_truth/', exist_ok=True)

train_path = 'mesh_dataset/dataset_3d/train/'
ground_truth_path = 'mesh_dataset/dataset_3d/ground_truth/'

for filename in tqdm(os.listdir(train_path)):
    if filename.endswith('.DS_Store'):
        continue
    else:
        mesh = trimesh.load(train_path + filename)
        voxel_matrix = get_voxel_matrix(mesh)
        if None in voxel_matrix:
            continue
        else:
            view = create_view(voxel_matrix)
            np.save('voxel_dataset/train/'+filename, view)
    
            truth_mesh = trimesh.load(train_path + filename)
            try:
                truth_voxel_matrix = get_voxel_matrix(mesh)
                truth_view = create_view(truth_voxel_matrix)
                np.save('voxel_dataset/ground_truth/'+filename, view)
            except:
                continue
    """

"os.makedirs('voxel_dataset/train/', exist_ok=True)\nos.makedirs('voxel_dataset/ground_truth/', exist_ok=True)\n\ntrain_path = 'mesh_dataset/dataset_3d/train/'\nground_truth_path = 'mesh_dataset/dataset_3d/ground_truth/'\n\nfor filename in tqdm(os.listdir(train_path)):\n    if filename.endswith('.DS_Store'):\n        continue\n    else:\n        mesh = trimesh.load(train_path + filename)\n        voxel_matrix = get_voxel_matrix(mesh)\n        if None in voxel_matrix:\n            continue\n        else:\n            view = create_view(voxel_matrix)\n            np.save('voxel_dataset/train/'+filename, view)\n    \n            truth_mesh = trimesh.load(train_path + filename)\n            try:\n                truth_voxel_matrix = get_voxel_matrix(mesh)\n                truth_view = create_view(truth_voxel_matrix)\n                np.save('voxel_dataset/ground_truth/'+filename, view)\n            except:\n                continue\n    "

In [ ]:
import os
import numpy as np
import trimesh
from joblib import Parallel, delayed
from tqdm import tqdm

def get_voxel_matrix(mesh):
    # Perform voxelization and convert to float32 directly
    voxel = mesh.voxelized(pitch=1.0 / 64)
    voxel_matrix = voxel.matrix.astype(np.float32)

    # Check dimensions of voxel_matrix
    if voxel_matrix.shape[0] > 256 or voxel_matrix.shape[1] > 256 or voxel_matrix.shape[2] > 256:
        return None

    # Initialize a padded matrix with zeros and copy voxel_matrix into it
    padded_matrix = np.zeros((256, 256, 256), dtype=np.float32)
    padded_matrix[:voxel_matrix.shape[0], :voxel_matrix.shape[1], :voxel_matrix.shape[2]] = voxel_matrix

    # Directly create the VoxelGrid from the padded matrix
    padded_voxelized = trimesh.voxel.VoxelGrid(padded_matrix)
    
    return padded_voxelized.matrix.astype(np.float32)

def create_view(voxel_matrix):
    # Use np.moveaxis for better performance instead of separate lists
    views = np.moveaxis(voxel_matrix, 0, -1)

    # Prepare the first_view with shape (256, 256, 256, 3)
    first_view = np.zeros((256, 256, 256, 3), dtype=np.float32)
    first_view[..., 0] = views[:, :, 0]  # Axial
    first_view[..., 1] = views[:, 0, :]  # Sagittal
    first_view[..., 2] = views[0, :, :]  # Coronal
    
    return first_view

import gc

def process_mesh_file(filename, train_path):
    try:
        # Load mesh and perform voxelization
        mesh = trimesh.load(os.path.join(train_path, filename))
        voxel_matrix = get_voxel_matrix(mesh)
        
        if voxel_matrix is None:
            return None, None

        # Create view from voxel matrix
        view = create_view(voxel_matrix)

        # Process ground truth similarly
        truth_mesh = trimesh.load(os.path.join(train_path, filename))
        truth_voxel_matrix = get_voxel_matrix(truth_mesh)
        
        if truth_voxel_matrix is None:
            return None, None

        truth_view = create_view(truth_voxel_matrix)

        # Free memory explicitly
        del mesh, voxel_matrix, truth_mesh, truth_voxel_matrix
        gc.collect()  # Force garbage collection
        return view, truth_view

    except Exception as e:
        print(f"Error processing {filename}: {e}")
        return None, None

def main():
    os.makedirs('voxel_dataset/train/', exist_ok=True)
    os.makedirs('voxel_dataset/ground_truth/', exist_ok=True)

    train_path = 'mesh_dataset/dataset_3d/train/'
    
    # List mesh files in the training directory
    mesh_files = [f for f in os.listdir(train_path) if not f.endswith('.DS_Store')]

    # Process mesh files in parallel
    results = Parallel(n_jobs=8)(delayed(process_mesh_file)(filename, train_path) for filename in tqdm(mesh_files))

    for filename, (view, truth_view) in zip(mesh_files, results):
        if view is not None and truth_view is not None:
            np.save(os.path.join('voxel_dataset/train/', filename), view)
            np.save(os.path.join('voxel_dataset/ground_truth/', filename), truth_view)

if __name__ == "__main__":
    main()

  0%|          | 56/11977 [01:42<7:11:45,  2.17s/it]/opt/conda/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
  1%|          | 120/11977 [03:34<5:31:18,  1.68s/it]

In [ ]:
print("done")